In [2]:
from datasets import load_dataset, concatenate_datasets
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import tqdm as notebook_tqdm

f:\Inlustro\t5speechenv\Lib\site-packages\transformers\utils\hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [1]:
import os
os.environ["HF_HOME"] = "F:/huggingface"
os.environ["HF_DATASETS_CACHE"] = "F:/huggingface/datasets"
os.environ["TRANSFORMERS_CACHE"] = "F:/huggingface/models"

In [3]:
ted_dataset = load_dataset("ted_talks_iwslt", "nl_en_2016", split="train", trust_remote_code=True)

subs_dataset = load_dataset("open_subtitles", "en-hi", split="train", trust_remote_code=True)

book_dataset = load_dataset("bookcorpus", split="train", trust_remote_code=True)

cnn_dataset = load_dataset("cnn_dailymail", "3.0.0", split="train", trust_remote_code=True)

In [4]:
def format_ted(example):
    return {
        "input": "Generate a professional speech about: " + example["translation"]["en"][:60],
        "target": example["translation"]["en"]
    }

def format_subs(example):
    return {
        "input": "Generate a funny speech about: " + example["translation"]["en"][:60],
        "target": example["translation"]["en"]
    }

def format_book(example):
    return {
        "input": "Generate a structured speech from: " + example["text"][:60],
        "target": example["text"]
    }

def format_cnn(example):
    return {
        "input": "Generate a formal speech about: " + example["article"][:60],
        "target": example["article"]
    }


In [8]:
ted_formatted = ted_dataset.map(format_ted)
subs_formatted = subs_dataset.map(format_subs)
book_formatted = book_dataset.map(format_book)
cnn_formatted = cnn_dataset.map(format_cnn)

In [9]:
# Only keep input and target
ted_formatted = ted_formatted.remove_columns([col for col in ted_formatted.column_names if col not in ["input", "target"]])
subs_formatted = subs_formatted.remove_columns([col for col in subs_formatted.column_names if col not in ["input", "target"]])
book_formatted = book_formatted.remove_columns([col for col in book_formatted.column_names if col not in ["input", "target"]])
cnn_formatted = cnn_formatted.remove_columns([col for col in cnn_formatted.column_names if col not in ["input", "target"]])



In [13]:
combined_dataset = concatenate_datasets([ted_formatted, subs_formatted, book_formatted, cnn_formatted])
for i, ds in enumerate(datasets_to_combine):
    print(f"Dataset {i} columns:", ds.column_names)
    print(f"Dataset {i} features:", ds.features)


Dataset 0 columns: ['input', 'target']
Dataset 0 features: {'input': Value(dtype='string', id=None), 'target': Value(dtype='string', id=None)}
Dataset 1 columns: ['input', 'target']
Dataset 1 features: {'input': Value(dtype='string', id=None), 'target': Value(dtype='string', id=None)}
Dataset 2 columns: ['input', 'target']
Dataset 2 features: {'input': Value(dtype='string', id=None), 'target': Value(dtype='string', id=None)}
Dataset 3 columns: ['input', 'target']
Dataset 3 features: {'input': Value(dtype='string', id=None), 'target': Value(dtype='string', id=None)}


In [14]:
print(combined_dataset)
print(combined_dataset[0])  # Just to see a sample



Dataset({
    features: ['input', 'target'],
    num_rows: 74388209
})
{'input': "Generate a professional speech about: We don't have to live in a world where 99 percent of rapists", 'target': 'We don\'t have to live in a world where 99 percent of rapists get away with it, says TED Fellow Jessica Ladd. With Callisto, a new platform for college students to confidentially report sexual assault, Ladd is helping survivors get the support and justice they deserve while respecting their privacy concerns. "We can create a world where there\'s a real deterrent to violating the rights of another human being," she says.'}


In [15]:
tokenizer = T5Tokenizer.from_pretrained("t5-large")

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
def tokenize(batch):
    inputs = tokenizer(batch["input"], padding="max_length", truncation=True, max_length=128)
    targets = tokenizer(batch["target"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = targets["input_ids"]  # Set the target input_ids as labels
    return inputs

In [ ]:
tokenized_datasets = combined_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/74388209 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_datasets[0])

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-large")

In [ ]:
training_args = TrainingArguments(
    output_dir="./t5-large-speech-gen",
    per_device_train_batch_size=1,                # 🧠 Safe for RTX 3050
    gradient_accumulation_steps=4,                # Simulates batch size 4
    num_train_epochs=3,
    learning_rate=3e-4,
    save_total_limit=2,
    save_steps=1000,
    logging_steps=500,
    evaluation_strategy="no",
    fp16=True,                                    # ✅ Mixed Precision
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

In [ ]:

model.save_pretrained("t5-large-speech-gen-model")
tokenizer.save_pretrained("t5-large-speech-gen-model")


In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-large-speech-gen-model")
tokenizer = T5Tokenizer.from_pretrained("t5-large-speech-gen-model")


In [ ]:
prompt = "Generate a funny speech about artificial intelligence in college life"


In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
output_ids = model.generate(inputs, max_length=200, num_beams=4, early_stopping=True)

In [ ]:
output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("\nGenerated Speech:\n")
print(output)

In [ ]:
print(combined_dataset)
print(combined_dataset[0])  # Just to see a sample



Dataset({
    features: ['input', 'target'],
    num_rows: 74388209
})
{'input': "Generate a professional speech about: We don't have to live in a world where 99 percent of rapists", 'target': 'We don\'t have to live in a world where 99 percent of rapists get away with it, says TED Fellow Jessica Ladd. With Callisto, a new platform for college students to confidentially report sexual assault, Ladd is helping survivors get the support and justice they deserve while respecting their privacy concerns. "We can create a world where there\'s a real deterrent to violating the rights of another human being," she says.'}


In [ ]:
print(combined_dataset)
print(combined_dataset[0])  # Just to see a sample



Dataset({
    features: ['input', 'target'],
    num_rows: 74388209
})
{'input': "Generate a professional speech about: We don't have to live in a world where 99 percent of rapists", 'target': 'We don\'t have to live in a world where 99 percent of rapists get away with it, says TED Fellow Jessica Ladd. With Callisto, a new platform for college students to confidentially report sexual assault, Ladd is helping survivors get the support and justice they deserve while respecting their privacy concerns. "We can create a world where there\'s a real deterrent to violating the rights of another human being," she says.'}


In [ ]:
print(combined_dataset)
print(combined_dataset[0])  # Just to see a sample



Dataset({
    features: ['input', 'target'],
    num_rows: 74388209
})
{'input': "Generate a professional speech about: We don't have to live in a world where 99 percent of rapists", 'target': 'We don\'t have to live in a world where 99 percent of rapists get away with it, says TED Fellow Jessica Ladd. With Callisto, a new platform for college students to confidentially report sexual assault, Ladd is helping survivors get the support and justice they deserve while respecting their privacy concerns. "We can create a world where there\'s a real deterrent to violating the rights of another human being," she says.'}


In [ ]:
print(combined_dataset)
print(combined_dataset[0])  # Just to see a sample



Dataset({
    features: ['input', 'target'],
    num_rows: 74388209
})
{'input': "Generate a professional speech about: We don't have to live in a world where 99 percent of rapists", 'target': 'We don\'t have to live in a world where 99 percent of rapists get away with it, says TED Fellow Jessica Ladd. With Callisto, a new platform for college students to confidentially report sexual assault, Ladd is helping survivors get the support and justice they deserve while respecting their privacy concerns. "We can create a world where there\'s a real deterrent to violating the rights of another human being," she says.'}


In [ ]:
print(combined_dataset)
print(combined_dataset[0])  # Just to see a sample



Dataset({
    features: ['input', 'target'],
    num_rows: 74388209
})
{'input': "Generate a professional speech about: We don't have to live in a world where 99 percent of rapists", 'target': 'We don\'t have to live in a world where 99 percent of rapists get away with it, says TED Fellow Jessica Ladd. With Callisto, a new platform for college students to confidentially report sexual assault, Ladd is helping survivors get the support and justice they deserve while respecting their privacy concerns. "We can create a world where there\'s a real deterrent to violating the rights of another human being," she says.'}


In [ ]:
print(combined_dataset)
print(combined_dataset[0])  # Just to see a sample



Dataset({
    features: ['input', 'target'],
    num_rows: 74388209
})
{'input': "Generate a professional speech about: We don't have to live in a world where 99 percent of rapists", 'target': 'We don\'t have to live in a world where 99 percent of rapists get away with it, says TED Fellow Jessica Ladd. With Callisto, a new platform for college students to confidentially report sexual assault, Ladd is helping survivors get the support and justice they deserve while respecting their privacy concerns. "We can create a world where there\'s a real deterrent to violating the rights of another human being," she says.'}
